# Hello Diffsol

This notebook demonstrates how to run a simple logistic decay ODE with `diffsol_pytorch.device`
and validate gradients.

In [ ]:
import math
import numpy as np
import torch
from diffsol_pytorch import DiffsolModule, device, testing
from helpers import describe_device, preferred_device, seed_everything

seed_everything(0)
device_target = preferred_device()
print(f"Running logistic ODE on {describe_device(device_target)}")

In [ ]:
LOGISTIC_CODE = """
in = [k]
k { 0.7 }
u { u = 1.0, }
F { -k * u, }
"""

module = DiffsolModule(LOGISTIC_CODE)
params = torch.tensor([0.7], dtype=torch.float64, device=device_target)
times = torch.linspace(0.0, 1.5, 64, dtype=torch.float64, device=device_target)
sol = device.solve_dense_tensor(module, params, times)
sol.shape

In [ ]:
def loss_fn(solution: np.ndarray):
    grad = np.zeros_like(solution)
    grad[0, -1] = 1.0
    return float(solution[0, -1]), grad

results = testing.check_gradients(LOGISTIC_CODE, [0.7], times.cpu().tolist(), loss_fn)
results["finite_difference"]
